<a href="https://colab.research.google.com/github/uk-zash/HuggingFace-Transformers/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [47]:
from datasets import load_dataset

dataset = load_dataset("glue" , "cola")
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [48]:
dataset["train"]["sentence"][0]

"Our friends won't buy this analysis, let alone the next one we propose."

In [49]:
dataset["train"]["label"][0]

1

In [50]:
from transformers import AutoTokenizer , TFAutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
model = TFAutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased" , num_labels = 1)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
def tokenize(data):

  return tokenizer(data["sentence"] , truncation = True)

dataset = dataset.map(tokenize , batched = True)

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

In [52]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
})

In [53]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer = tokenizer , return_tensors = "tf")

In [54]:
dataset.set_format(type='tensorflow', columns=['input_ids', 'attention_mask', 'label'])
train_dataset = model.prepare_tf_dataset (dataset["train"]  ,
                                      batch_size = 16 ,
                                      shuffle = True ,
                                      collate_fn = data_collator)

In [55]:

test_dataset = model.prepare_tf_dataset(dataset["validation"],
                                        batch_size = 16 ,
                                        shuffle= True ,
                                        collate_fn = data_collator)

In [39]:
!pip install huggingface_hub

In [41]:
from huggingface_hub import notebook_login
notebook_login()

In [68]:

model.compile(optimizer = "adam", loss = "binary_crossentropy" , metrics = ["accuracy"])

In [76]:
from transformers import PushToHubCallback

push_to_hub = PushToHubCallback(
    output_dir="Cola_finetuned",
    tokenizer=tokenizer,
    hub_model_id="ukzash1/FineTuned_Cola",
    save_steps=1000,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/Cola_finetuned is already a clone of https://huggingface.co/ukzash1/FineTuned_Cola. Make sure you pull the latest changes with `repo.git_pull()`.


In [77]:
history = model.fit(
    train_dataset ,
    validation_data = test_dataset ,
    epochs = 3,
    callbacks = [push_to_hub]
)

Epoch 1/3
534/534 [==============================] - 73s 136ms/step - loss: 4.5066 - accuracy: 0.7045 - val_loss: 4.7067 - val_accuracy: 0.6913
Epoch 2/3
534/534 [==============================] - 79s 149ms/step - loss: 4.5084 - accuracy: 0.7044 - val_loss: 4.6921 - val_accuracy: 0.6923
Epoch 3/3
534/534 [==============================] - 73s 136ms/step - loss: 4.5102 - accuracy: 0.7042 - val_loss: 4.7067 - val_accuracy: 0.6913


In [78]:
from transformers import pipeline
classifier = pipeline("text-classification", model="ukzash1/FineTuned_Cola")

config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/434M [00:00<?, ?B/s]

Some layers from the model checkpoint at ukzash1/FineTuned_Cola were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at ukzash1/FineTuned_Cola.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [79]:
classifier("This is great")

[{'label': 'LABEL_0', 'score': 0.9978947043418884}]